Creando Endpoint de Analisis de sentimiento

In [1]:
import os, sys
import json

ENDPOINT_ID = os.getenv("ENDPOINT_CONFIG_ID") # Identificador unico
ENDPOINT_COMPANY = os.getenv("ENDPOINT_CONFIG_COMPANY", "Empresa") # Nombre de la compañia
ENDPOINT_COMPANY_ABBR = os.getenv("ENDPOINT_CONFIG_COMPANY_ABBR", "E") # Abreviatura de la compañia

ENDPOINT_DISPLAY_NAME = os.getenv("ENDPOINT_DISPLAY_NAME", "myENDPOINT")
ENDPOINT_DESCRIPTION = os.getenv("ENDPOINT_DESCRIPTION", "Example of my first end-point")

ENDPOINT_SERVICE_ACCOUNT = os.getenv("ENDPOINT_SERVICE_ACCOUNT") # Cuenta de Servicio que se asignara al ENDPOINT
ENDPOINT_PROJECT_ID = os.getenv("ENDPOINT_PROJECT_ID", "proyectodatapath1") # Proyecto GCP donde se ejecuta el ENDPOINT
ENDPOINT_REGION = os.getenv("ENDPOINT_REGION", "us-central1") # Region
ENDPOINT_BUCKET = os.getenv("ENDPOINT_BUCKET", "bucketdatapath") # Bucket donde se guardaran artefactos del despliegue

ENDPOINT_PATH_ROOT = os.getenv("ENDPOINT_PATH_ROOT", "endpoint")
ENDPOINT_MODEL_PATH_ROOT = os.getenv("ENDPOINT_MODEL_PATH_ROOT", "endpoint/model")

In [2]:
PROJECT_ID          = ENDPOINT_PROJECT_ID
REGION              = ENDPOINT_REGION
#BUCKET_NAME         = "gs://" + ENDPOINT_BUCKET 
BUCKET_NAME         = ENDPOINT_BUCKET
MODEL_NAME          = 'sentimientos.h5'
SAVED_MODEL_DIR    = 'saved_model'
SERVICE_ACCOUNT     = ENDPOINT_SERVICE_ACCOUNT

In [3]:
#ENDPOINT_ROOT = os.path.join(BUCKET_NAME, ENDPOINT_PATH_ROOT)
ENDPOINT_MODEL_ROOT = os.path.join(ENDPOINT_MODEL_PATH_ROOT, MODEL_NAME)
ENDPOINT_SAVED_MODEL_ROOT = os.path.join(ENDPOINT_MODEL_PATH_ROOT, SAVED_MODEL_DIR)
#print(ENDPOINT_ROOT)
print(ENDPOINT_SAVED_MODEL_ROOT)

endpoint/model/saved_model


In [5]:
!pip install tensorflow

In [9]:
import tensorflow as tf
from tensorflow import keras

In [10]:
# Cargar el modelo
from tensorflow.keras.models import load_model
model = load_model('sentimientos.h5')

In [11]:
# Exportar el modelo a saved_model

tf.saved_model.save(model, './saved_model')

INFO:tensorflow:Assets written to: ./saved_model/assets


INFO:tensorflow:Assets written to: ./saved_model/assets


In [12]:
from google.cloud import storage

def upload_to_gcs(bucket_name, local_file_path, destination_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(local_file_path)

# Especifica el nombre del bucket de GCS y los nombres de archivo
bucket_name = BUCKET_NAME#ENDPOINT_MODEL_ROOT#'nombre_del_bucket'
local_file_path = MODEL_NAME
destination_blob_name = ENDPOINT_MODEL_ROOT

# Sube el archivo al bucket de GCS
upload_to_gcs(bucket_name, local_file_path, destination_blob_name)

In [13]:
import os
from google.cloud import storage

def upload_directory_to_gcs(bucket_name, local_directory, destination_directory):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    for root, dirs, files in os.walk(local_directory):
        for file in files:
            local_path = os.path.join(root, file)
            destination_path = os.path.join(destination_directory, os.path.relpath(local_path, local_directory))
            blob = bucket.blob(destination_path)
            blob.upload_from_filename(local_path)
            print(f'Subido: {local_path} -> gs://{bucket_name}/{destination_path}')

# Especifica el nombre del bucket de GCS y los directorios local y de destino
bucket_name = BUCKET_NAME
local_directory = SAVED_MODEL_DIR
destination_directory = ENDPOINT_SAVED_MODEL_ROOT

# Sube el directorio al bucket de GCS
upload_directory_to_gcs(bucket_name, local_directory, destination_directory)


Subido: saved_model/fingerprint.pb -> gs://bucketdatapath/endpoint/model/saved_model/fingerprint.pb
Subido: saved_model/saved_model.pb -> gs://bucketdatapath/endpoint/model/saved_model/saved_model.pb
Subido: saved_model/variables/variables.data-00000-of-00001 -> gs://bucketdatapath/endpoint/model/saved_model/variables/variables.data-00000-of-00001
Subido: saved_model/variables/variables.index -> gs://bucketdatapath/endpoint/model/saved_model/variables/variables.index


In [14]:
!pip install -U google-cloud-aiplatform "shapely<2"

  Using cached google_cloud_aiplatform-1.36.4-py2.py3-none-any.whl.metadata (27 kB)
Using cached google_cloud_aiplatform-1.36.4-py2.py3-none-any.whl (3.3 MB)
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.0.0
    Uninstalling google-cloud-aiplatform-1.0.0:
      Successfully uninstalled google-cloud-aiplatform-1.0.0


In [22]:
from google.cloud import aiplatform
from typing import Optional

def upload_model_sample(
    project: str,
    location: str,
    display_name: str,
    serving_container_image_uri: str,
    artifact_uri: Optional[str] = None
):

    aiplatform.init(project=project, location=location)
    
    model = aiplatform.Model.upload(
        display_name=display_name,
        artifact_uri=artifact_uri,
        serving_container_image_uri=serving_container_image_uri,
    )
    
    model.wait()

    print(model.display_name)
    print(model.resource_name)
    return model

In [19]:
print(destination_directory)

endpoint/model/saved_model


In [16]:
#BUCKET_NAME         = "gs://" + ENDPOINT_BUCKET 
ARTIFACT_URI           = "gs://" + os.path.join(ENDPOINT_BUCKET, destination_directory)
print(ARTIFACT_URI)

gs://bucketdatapath/endpoint/model/saved_model


In [24]:
#upload_model_sample(project= PROJECT_ID, location = REGION, display_name= 'keras_registry', serving_container_image_uri = destination_directory, artifact_uri = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-11:latest')
#'gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-5:latest'

keras_model_registry = upload_model_sample(
    project=PROJECT_ID,
    location=REGION,
    display_name='sentimientos_datapath',
    serving_container_image_uri='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-11:latest', # URI de la imagen del contenedor
    artifact_uri=ARTIFACT_URI # Cambia esto a la ruta correcta de GCS
)

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/759220441436/locations/us-central1/models/1066805554896175104/operations/384830650269564928
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/759220441436/locations/us-central1/models/1066805554896175104@1
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/759220441436/locations/us-central1/models/1066805554896175104@1')


sentimientos_datapath
projects/759220441436/locations/us-central1/models/1066805554896175104


In [25]:
name_model_registry = keras_model_registry.name

In [26]:
print(name_model_registry)

1066805554896175104


In [27]:
def create_endpoint_sample(
    project: str,
    display_name: str,
    location: str,
):
    aiplatform.init(project=project, location=location)

    endpoint = aiplatform.Endpoint.create(
        display_name=display_name,
        project=project,
        location=location,
    )

    print(endpoint.display_name)
    print(endpoint.resource_name)
    return endpoint

In [28]:
endpoint = create_endpoint_sample(project= PROJECT_ID, \
                                    display_name= 'sentimientos_datapath', \
                                    location= REGION)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/759220441436/locations/us-central1/endpoints/1759062473790455808/operations/3771537570052177920
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/759220441436/locations/us-central1/endpoints/1759062473790455808
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/759220441436/locations/us-central1/endpoints/1759062473790455808')


sentimientos_datapath
projects/759220441436/locations/us-central1/endpoints/1759062473790455808


In [29]:
ENDPOINT_NAME = 'sentimientos_datapath'
!gcloud ai endpoints list \
  --region=$REGION\
  --filter=display_name=$ENDPOINT_NAME

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
ENDPOINT_ID          DISPLAY_NAME
1759062473790455808  sentimientos_datapath


In [30]:
!pip install shap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 12.9 MB/s eta 0:00:0000:01


In [31]:
from typing import Dict, Sequence, Tuple
#import shap.explain

def deploy_model_with_dedicated_resources_sample(
    project,
    location,
    model_name: str,
    machine_type: str,
    endpoint: Optional[aiplatform.Endpoint] = None,
    deployed_model_display_name: Optional[str] = None,
    traffic_percentage: Optional[int] = 0,
    traffic_split: Optional[Dict[str, int]] = None,
    min_replica_count: int = 1,
    max_replica_count: int = 1,
    accelerator_type: Optional[str] = None,
    accelerator_count: Optional[int] = None,
    #explanation_metadata: Optional[explain.ExplanationMetadata] = None,
    #explanation_parameters: Optional[explain.ExplanationParameters] = None,
    metadata: Optional[Sequence[Tuple[str, str]]] = (),
    sync: bool = True,
):
    """
    model_name: A fully-qualified model resource name or model ID.
          Example: "projects/123/locations/us-central1/models/456" or
          "456" when project and location are initialized or passed.
    """

    aiplatform.init(project=project, location=location)
    print('model name: ', model_name)
    model = aiplatform.Model(model_name=model_name)
    #model = aiplatform.Model(display_name=model_name)
    
    # The explanation_metadata and explanation_parameters should only be
    # provided for a custom trained model and not an AutoML model.
    model.deploy(
        endpoint=endpoint,
        deployed_model_display_name=deployed_model_display_name,
        traffic_percentage=traffic_percentage,
        traffic_split=traffic_split,
        machine_type=machine_type,
        min_replica_count=min_replica_count,
        max_replica_count=max_replica_count,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        #explanation_metadata=explanation_metadata,
        #explanation_parameters=explanation_parameters,
        metadata=metadata,
        sync=sync,
    )
    
    model.wait()

    print(model.display_name)
    print(model.resource_name)
    return model

In [33]:
deploy_model_with_dedicated_resources_sample(
    project=PROJECT_ID, \
    location=REGION, \
    model_name= name_model_registry, \
    machine_type='n1-standard-2', \
    endpoint= endpoint, \
    deployed_model_display_name='sentimientos_datapath_endpoint', \
    #traffic_percentage: Optional[int] = 0,
    #traffic_split: Optional[Dict[str, int]] = None,
    min_replica_count= 1, \
    max_replica_count= 1
    )

model name:  1066805554896175104


INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/759220441436/locations/us-central1/endpoints/1759062473790455808
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/759220441436/locations/us-central1/endpoints/1759062473790455808/operations/6003071185414258688
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/759220441436/locations/us-central1/endpoints/1759062473790455808


sentimientos_datapath
projects/759220441436/locations/us-central1/models/1066805554896175104


resource name: projects/759220441436/locations/us-central1/models/1066805554896175104

In [34]:
def endpoint_predict_sample(
    project: str, location: str, instances: list, endpoint: str
):
    aiplatform.init(project=project, location=location)

    endpoint = aiplatform.Endpoint(endpoint)

    prediction = endpoint.predict(instances=instances)
    print(prediction)
    return prediction

In [35]:
print(endpoint.name)

1759062473790455808


In [38]:
import numpy as np

In [39]:
input_data = np.array([[
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 52, 0, 1,
    1641, 8, 11, 3753
]])

In [41]:
# Convierte el array de NumPy en una lista de Python
instances = input_data.tolist()

# Ahora puedes pasar 'instances' a la función de predicción
response = endpoint_predict_sample(
    project=PROJECT_ID,
    location=REGION,
    instances=instances,  # Asegúrate de pasar la lista aquí
    endpoint=endpoint.name
)


Prediction(predictions=[[0.586094499]], deployed_model_id='3228243104964280320', model_version_id='1', model_resource_name='projects/759220441436/locations/us-central1/models/1066805554896175104', explanations=None)


In [42]:
print("Predicción del Endpoint: \n", response.predictions)

Predicción del Endpoint: 
 [[0.586094499]]


In [43]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb

max_features = 5000  # Número máximo de palabras a considerar como características
max_len = 200  # Longitud máxima de la secuencia de texto

# Cargar el word_index
word_index = imdb.get_word_index()

# Función para convertir texto a secuencia
def text_to_sequence(text, word_index, max_features, max_len):
    tokens = text.lower().split()
    sequence = [word_index.get(word, 0) if word_index.get(word, 0) < max_features else 0 for word in tokens]
    padded_sequence = pad_sequences([sequence], maxlen=max_len)
    return padded_sequence

# Texto de entrada para la predicción
sample_review = "this movie is bitter I don't like it"

# Convertir el texto en secuencia
sample_sequence = text_to_sequence(sample_review, word_index, max_features, max_len)
print(sample_sequence)

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0   11   17   

In [44]:
instances = sample_sequence.tolist()

# Ahora puedes pasar 'instances' a la función de predicción
response = endpoint_predict_sample(
    project=PROJECT_ID,
    location=REGION,
    instances=instances,  # Asegúrate de pasar la lista aquí
    endpoint=endpoint.name
)

Prediction(predictions=[[0.304122239]], deployed_model_id='3228243104964280320', model_version_id='1', model_resource_name='projects/759220441436/locations/us-central1/models/1066805554896175104', explanations=None)


In [45]:
print("Predicción del Endpoint: \n", response.predictions)

Predicción del Endpoint: 
 [[0.304122239]]
